In [16]:
import pandas as pd

In [17]:
data = pd.read_csv('model_input_data/from_combined/com_original_data.csv')
data.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,WHIP,LOB,ERA,FIP,...,vs_PF,vs_H_A,vs_WLS,vs_PITCHER_INNING,vs_PITCHER_ER,vs_HITTER_AB,vs_HITTER_HIT,wls,vs_wls,YEAR
0,20160401HHLG0,20160401,LG,HH,0,B,1.333,0.750,3.000,7.333,...,939,0,0,11.333,4,46,13,W,L,2016
1,20160401HHLG0,20160401,HH,LG,0,T,1.147,0.690,3.176,23.059,...,939,1,1,12.000,4,42,9,L,W,2016
2,20160401HTNC0,20160401,NC,HT,0,B,1.222,0.755,4.000,22.778,...,1035,0,0,8.000,5,34,8,W,L,2016
3,20160401HTNC0,20160401,HT,NC,0,T,1.750,0.804,5.625,38.750,...,1035,1,1,9.000,4,30,9,L,W,2016
4,20160401KTSK0,20160401,KT,SK,0,T,1.111,0.698,4.000,11.444,...,971,1,0,9.000,8,36,10,W,L,2016


In [39]:
data.columns

Index(['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'WHIP',
       'LOB', 'ERA', 'FIP', 'H_9', 'K_9', 'BB_9', 'HR_9', 'oAVG', 'oSLG',
       'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT', 'AVG', 'OBP', 'SLG',
       'OPS', 'RC', 'GPA', 'BA', 'SECA', 'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT',
       'DER', 'PF', 'H_A', 'WLS', 'PITCHER_INNING', 'PITCHER_ER', 'HITTER_AB',
       'HITTER_HIT', 'vs_WHIP', 'vs_LOB', 'vs_ERA', 'vs_FIP', 'vs_H_9',
       'vs_K_9', 'vs_BB_9', 'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP',
       'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT', 'vs_AVG', 'vs_OBP',
       'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 'vs_TA',
       'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER', 'vs_PF',
       'vs_H_A', 'vs_WLS', 'vs_PITCHER_INNING', 'vs_PITCHER_ER',
       'vs_HITTER_AB', 'vs_HITTER_HIT', 'wls', 'vs_wls', 'YEAR'],
      dtype='object')

# 1. 연도-팀별 key(24)경기 기준 통계
#### 이때, 데이터 늘리기를 적용
- train, test를 미리 나누어 둔 후, train 부분에 대해서만 데이터 늘리기 적용

In [18]:
data_col = list(data.columns)
team = list(data.T_ID.unique())

default_col = ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID',"YEAR"]
size_col = ['wls','vs_wls']

sum_col = ['HEADER_NO',
           'PITCHER_INNING', 'PITCHER_ER', 'HITTER_AB','HITTER_HIT',  
           'vs_PITCHER_INNING', 'vs_PITCHER_ER','vs_HITTER_AB', 'vs_HITTER_HIT']

tmp = set(default_col)|set(size_col)|set(sum_col)
mean_col = list(set(data_col)-tmp)

In [19]:
size_df = data[default_col + size_col]
sum_df = data[default_col + sum_col]
mean_df = data[default_col + mean_col]

###  1) size

In [20]:
def make_pct(df):
    tmp = pd.DataFrame({"COUNT":df.groupby(["T_ID","wls","YEAR"]).size()}).pivot_table("COUNT",["T_ID","YEAR"],"wls").reset_index().sort_values(["YEAR","T_ID"]).reset_index(drop=True)
    tmp.fillna(0,inplace= True)

    if "L" not in list(tmp.columns):
        tmp["L"] = 0

    if "W" not in list(tmp.columns):
        tmp["W"] = 0
    
    tmp["PCT"] = tmp['W']/(tmp['W']+tmp['L'])
    return tmp

In [21]:
size_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019,2020]:
    print("===",y)
    
    y_tmp = size_df[size_df["YEAR"]==y]
    for t in team:
        print(t)
        
        idx = 0
        
        ttmp_df = y_tmp[y_tmp["T_ID"]==t]
        tmp_size = len(ttmp_df)
        
        #test
        test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
        tmp2 = make_pct(test_tmp_df)
        tmp2['IDX'] = 777
        tmp2['MERGE_IDX'] = 0

        size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)
        
        
        # train
        tmp_df = ttmp_df.iloc[:tmp_size-key,:]
        t_size = len(tmp_df)
               
        m_idx = 1
        while idx < key:
            k_idx = 1
            for k in range((t_size-idx)//key):
                tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                tmp2 = make_pct(tmp1)
                
                tmp2['IDX'] = k_idx 
                tmp2['MERGE_IDX'] = m_idx
                k_idx += 1
                m_idx += 1
                
                size_df_cal = pd.concat([size_df_cal, tmp2], axis = 0)
            idx += 1
            

# size_df_cal.drop(["W","D","L"],axis = 1, inplace = True)

=== 2016
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2017
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2018
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2019
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2020
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS


In [22]:
size_df_cal

,T_ID,YEAR,D,L,W,PCT,IDX,MERGE_IDX
0,LG,2016,1.0,9,14,0.608696,777,0
0,LG,2016,NaN,12,12,0.500000,1,1
0,LG,2016,1.0,12,11,0.478261,2,2
0,LG,2016,NaN,15,9,0.375000,3,3
0,LG,2016,NaN,13,11,0.458333,4,4
...,...,...,...,...,...,...,...,...
0,SS,2020,NaN,10,14,0.583333,1,14
0,SS,2020,NaN,10,14,0.583333,1,15
0,SS,2020,NaN,10,14,0.583333,1,16
0,SS,2020,NaN,10,14,0.583333,1,17


### 2) sum

In [23]:
def make_sum(df):
    tmp = df.groupby(["T_ID","YEAR"]).sum().reset_index()
    
    # ERA
    tmp["era"] = tmp["PITCHER_ER"]*9/tmp["PITCHER_INNING"]
    tmp["vs_era"] = tmp["vs_PITCHER_ER"]*9/tmp["vs_PITCHER_INNING"]
    
    #AVG
    tmp['avg'] = tmp["HITTER_HIT"]/tmp["HITTER_AB"]
    tmp['vs_avg'] = tmp["vs_HITTER_HIT"]/tmp["vs_HITTER_AB"]
    
    re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"]
#     re_col = ["T_ID","YEAR","HEADER_NO","era","avg","vs_era","vs_avg"] + vsteam_col
    return tmp[re_col]
    

In [24]:
sum_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019,2020]:
    print("===",y)
    
    y_tmp = sum_df[sum_df["YEAR"]==y]
    for t in team:
        print(t)
        
        idx = 0
        
        ttmp_df = y_tmp[y_tmp["T_ID"]==t]
        tmp_size = len(ttmp_df)
        
        #test
        test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
        tmp2 = make_sum(test_tmp_df)
        tmp2['IDX'] = 777
        tmp2['MERGE_IDX'] = 0

        sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
        
        
        # train
        tmp_df = ttmp_df.iloc[:tmp_size-key,:]
        t_size = len(tmp_df)
               
        m_idx = 1
        while idx < key:
            k_idx = 1
            for k in range((t_size-idx)//key):
                tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                tmp2 = make_sum(tmp1)
                
                tmp2['IDX'] = k_idx 
                tmp2['MERGE_IDX'] = m_idx
                k_idx += 1
                m_idx += 1
                
                sum_df_cal = pd.concat([sum_df_cal, tmp2], axis = 0)
            idx += 1

=== 2016
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2017
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2018
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2019
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2020
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS


In [25]:
sum_df_cal

,T_ID,YEAR,HEADER_NO,era,avg,vs_era,vs_avg,IDX,MERGE_IDX
0,LG,2016,0,3.649770,0.296069,4.993723,0.268530,777,0
0,LG,2016,0,5.358453,0.253317,4.555222,0.298349,1,1
0,LG,2016,0,4.715603,0.291169,4.659483,0.274605,2,2
0,LG,2016,0,5.584768,0.305842,5.148837,0.288889,3,3
0,LG,2016,0,5.551402,0.297110,5.833333,0.288942,4,4
...,...,...,...,...,...,...,...,...,...
0,SS,2020,3,3.693396,0.263694,5.003187,0.252488,1,14
0,SS,2020,3,3.778302,0.265997,5.151025,0.254975,1,15
0,SS,2020,3,4.009479,0.270338,5.193950,0.261462,1,16
0,SS,2020,3,3.685714,0.265743,5.065175,0.257783,1,17


### 3) mean

In [26]:
def make_mean(df):
    tmp = df.groupby(["T_ID","YEAR"]).mean().reset_index()
    tmp.drop(["GDAY_DS"],axis=1,inplace = True)
    return tmp

In [27]:
mean_df_cal = pd.DataFrame([])

key = 24
for y in [2016,2017,2018,2019,2020]:
    print("===",y)
    
    y_tmp = mean_df[mean_df["YEAR"]==y]
    for t in team:
        print(t)
        
        idx = 0
        
        ttmp_df = y_tmp[y_tmp["T_ID"]==t]
        tmp_size = len(ttmp_df)
        
        #test
        test_tmp_df = ttmp_df.iloc[tmp_size-key:,:]
        tmp2 = make_mean(test_tmp_df)
        tmp2['IDX'] = 777
        tmp2['MERGE_IDX'] = 0

        mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
        
        
        # train
        tmp_df = ttmp_df.iloc[:tmp_size-key,:]
        t_size = len(tmp_df)
               
        m_idx = 1
        while idx < key:
            k_idx = 1
            for k in range((t_size-idx)//key):
                tmp1 = tmp_df.iloc[key*k+idx:key*(k+1)+idx,:]
                tmp2 = make_mean(tmp1)
                
                tmp2['IDX'] = k_idx 
                tmp2['MERGE_IDX'] = m_idx
                k_idx += 1
                m_idx += 1
                
                mean_df_cal = pd.concat([mean_df_cal, tmp2], axis = 0)
            idx += 1


=== 2016
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2017
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2018
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2019
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS
=== 2020
LG
HH
NC
HT
KT
SK
WO
LT
OB
SS


In [28]:
mean_df_cal 

,T_ID,YEAR,BA,vs_P2_WHIP_RT,vs_BB_9,vs_WHIP,vs_K_9,oAVG,vs_OPS,P_WHIP_RT,...,FIP,vs_BA,DER,vs_OBP,vs_XR,CB_WHIP_RT,P_HRA_RT,vs_SLG,IDX,MERGE_IDX
0,LG,2016,0.288292,1.609917,3.927083,1.588458,6.367167,0.263917,0.714208,1.115792,...,19.183417,0.265625,0.698875,0.335333,4.364250,1.740292,0.280875,0.378875,777,0
0,LG,2016,0.245625,1.544792,4.053000,1.417625,7.958750,0.286875,0.777333,2.098125,...,20.730708,0.289208,0.659333,0.362375,5.260250,1.918625,0.216875,0.414958,1,1
0,LG,2016,0.286125,1.626833,3.439667,1.526958,6.593083,0.271625,0.717458,2.351792,...,18.914375,0.268792,0.696458,0.350292,4.609000,2.001417,0.279667,0.367167,2,2
0,LG,2016,0.301250,1.630458,3.305583,1.615917,6.863167,0.288208,0.800958,1.840250,...,27.979500,0.285500,0.687208,0.361542,5.693083,1.690333,0.249625,0.439417,3,3
0,LG,2016,0.292292,2.127708,4.143000,1.644917,6.665708,0.284042,0.804208,1.580875,...,27.325833,0.280625,0.685417,0.365458,5.810750,1.630417,0.290625,0.438750,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,SS,2020,0.258500,1.280875,3.359375,1.373667,6.964333,0.245333,0.688375,1.421375,...,22.090208,0.243208,0.726125,0.320333,4.296250,1.742333,0.263208,0.368042,1,14
0,SS,2020,0.261458,1.324625,3.437500,1.402583,7.011208,0.247917,0.696542,1.390125,...,22.409625,0.245792,0.725542,0.320667,4.351000,1.742333,0.262042,0.375875,1,15
0,SS,2020,0.267000,1.349042,3.395833,1.416458,6.886208,0.254792,0.709542,1.421375,...,22.692042,0.252083,0.719417,0.327792,4.509000,1.846500,0.269000,0.381750,1,16
0,SS,2020,0.262417,1.316333,3.520833,1.407208,7.011208,0.251833,0.707208,1.371375,...,22.289250,0.249083,0.718333,0.322750,4.357417,1.891167,0.258000,0.384458,1,17


# 2. 데이터 합치기

In [32]:
import numpy as np

In [33]:
tmp_table = pd.merge(size_df_cal, sum_df_cal, how= 'left', left_on = ['T_ID',"YEAR","MERGE_IDX","IDX"], right_on  = ['T_ID',"YEAR","MERGE_IDX","IDX"])
merge_df = pd.merge(tmp_table, mean_df_cal, how = 'left', left_on = ['T_ID',"YEAR","MERGE_IDX","IDX"], right_on  = ['T_ID',"YEAR","MERGE_IDX","IDX"] )

merge_df["D"] = merge_df["D"].fillna(0).astype(np.int64)
# merge_df.drop(["MERGE_IDX"],axis=1,inplace = True)

In [35]:
merge_df.to_csv("augment_24WLS.csv", index = False)

In [34]:
merge_df 

,T_ID,YEAR,D,L,W,PCT,IDX,MERGE_IDX,HEADER_NO,era,...,vs_AVG,vs_SECA,FIP,vs_BA,DER,vs_OBP,vs_XR,CB_WHIP_RT,P_HRA_RT,vs_SLG
0,LG,2016,1,9,14,0.608696,777,0,0,3.649770,...,0.265625,0.284625,19.183417,0.265625,0.698875,0.335333,4.364250,1.740292,0.280875,0.378875
1,LG,2016,0,12,12,0.500000,1,1,0,5.358453,...,0.289208,0.312042,20.730708,0.289208,0.659333,0.362375,5.260250,1.918625,0.216875,0.414958
2,LG,2016,1,12,11,0.478261,2,2,0,4.715603,...,0.268792,0.296458,18.914375,0.268792,0.696458,0.350292,4.609000,2.001417,0.279667,0.367167
3,LG,2016,0,15,9,0.375000,3,3,0,5.584768,...,0.285500,0.344042,27.979500,0.285500,0.687208,0.361542,5.693083,1.690333,0.249625,0.439417
4,LG,2016,0,13,11,0.458333,4,4,0,5.551402,...,0.280625,0.379125,27.325833,0.280625,0.685417,0.365458,5.810750,1.630417,0.290625,0.438750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095,SS,2020,0,10,14,0.583333,1,14,3,3.693396,...,0.243208,0.320625,22.090208,0.243208,0.726125,0.320333,4.296250,1.742333,0.263208,0.368042
4096,SS,2020,0,10,14,0.583333,1,15,3,3.778302,...,0.245792,0.325833,22.409625,0.245792,0.725542,0.320667,4.351000,1.742333,0.262042,0.375875
4097,SS,2020,0,10,14,0.583333,1,16,3,4.009479,...,0.252083,0.328333,22.692042,0.252083,0.719417,0.327792,4.509000,1.846500,0.269000,0.381750
4098,SS,2020,0,10,14,0.583333,1,17,3,3.685714,...,0.249083,0.330667,22.289250,0.249083,0.718333,0.322750,4.357417,1.891167,0.258000,0.384458
